# Textbook RSA Solution

## Exploring our Public Keys

As we first approach this problem, we see that $\texttt{ciphertexts.csv}$ lists $100$ different public keys for the same message, where $c_i$ is our ciphertext in decimal form, $n_i$ is our public modulus, and $e_i$ is our encrypting exponent. Further inspection into $\texttt{scrypt.py}$ reveals that the custom RSA scheme makes no use of random padding to secure messages (RSA without padding is known as textbook RSA, hence the challenge name).  Given there is no randomness added to the same message and the same exponent is used to encrypt each method, we may try two avenues of attack: find a common prime or employ the Chinese Remainder Theorem. Whatever the case, we start by collecting our data:

In [1]:
f = open('ciphertexts.csv')
public = f.read()[:-1] #Omit last linebreak character
f.close()
rows = public.split('\n')

ciphers = []
moduli = []

for row in rows[1:]: #Skip first line with titles
    id_, c, n, e = row.split(',')
    ciphers.append(int(c))
    moduli.append(int(n))


## Attempting to Find a Common Prime

Prime estimating functions approximate there are over $10^{150}$ $512$-bit primes, so finding a common prime in this setting (or similar ones) will almost never happen. However, to use the Chinese Remainder Theorem, we must check that multiple of our public moduli are pairwise coprime. Still, there is very little computational cost incurred by checking all $100$ moduli are pairwise coprime thanks to the Euclidean algorithm. In fact, some languages may have this algorithm already incorporated with built-in functions (here with Python, we use the $\texttt{gcd}$ function from the internal library $\texttt{math}$ to check if two integers are coprime). Further, we use $\texttt{combinations}$ from $\texttt{itertools}$ to focus calculations on only the $4950$ unique unorders pairs of our $100$ moduli. Nevertheless, we find all our public moduli are pairwise coprime:

In [2]:
from math import gcd
from itertools import combinations

common_prime = False
for combo in combinations(moduli, 2):
    d = gcd(*combo)
    if d != 1:
        common_prime = True
        break

if not common_prime:
    print('Moduli are pairwise coprime!')

Moduli are pairwise coprime!


## The Chinese Remainder Theorem

Here, we state the Theorem and it corresponding proof. It is worth mentioning the proof is mainly provided to supplement insight into how we (and other sources) build a custom algorithm to solve a system of equations involving modular arthimetic. Still, the reader would not be faulted for skipping the proof and are safe to do so as many online sources supply code to approach the Chinese Remainder Theorem.

**Theorem.** Let $n_1, n_2, \cdots, n_k$ be integers greater than 1 and $N$ be the product of all $n_i$. That is,

$$N = n_1n_2\cdots n_k.$$

If $\gcd(n_i, n_j) = 1$ whenever $i \ne j$, and if $c_1, c_2, \cdots, c_k$ are integers such that $0 \le c_i < n_i$ for every $i$, then there is a unique integer $x$ such that $0 \le x < N$ and $x \equiv c_i\ \mathrm{(mod\ } n_i)$ for each $i$.

*Proof.* We start with existence. Let $N_i = N/n_i$ (note that $N_i$ is an integer as $n_i$ is a factor of $N$). Given $\gcd(n_i, n_j) = 1$ whenever $i \ne j$, we have $\gcd(n_i, N_i) = 1$ for each $i$. In words, $n_i$ is coprime with integer $N_i$. By Bezout's Identity, there exists integers $q_i$ and $M_i$ such that

$$
\begin{align}
q_in_i + M_i(N_i) &= 1 \\
M_i(N_i) &= (-q_i)n_i + 1.
\end{align}
$$

That is to say, there exists an integer $M_i$ such that $M_iN_i \equiv 1\ \ (\mathrm{mod}\ n_i)$. Let 
$$x = c_1M_1N_1 + c_2M_2N_2 + \cdots + c_kM_kN_k.$$

Seeing that $n_1$ appears in the factorization of every integer $N_i$ except when $i = 1$, we have

$$
\begin{align}
x &\equiv c_1M_1N_1 + c_2M_2(0) + \cdots + c_kM_k(0)\ \ (\mathrm{mod}\ n_1) \\
&\equiv c_1M_1N_1\ \ (\mathrm{mod}\ n_1) \\
&\equiv c_1(1)\ \ (\mathrm{mod}\ n_1) \\
&\equiv c_1\ \ (\mathrm{mod}\ n_1).
\end{align}
$$

A similar argument shows $x \equiv c_i\ \ (\mathrm{mod}\ n_i)$ for each $i$. If $x < 0$ or $N < x$, choose the remainder of $x$ after Euclidean division by $N$ instead.

To show $x$ is unique, suppose $x' = c_i\ \ (\mathrm{mod}\ n_i)$ for each $i$ with $0 \le x' < N$. That is, $x = an_i + c_i$ and $x' = a'n_i + c_i$. Without loss of generality, assume $x \ge x'$. Then $x - x' = (a - a')n_i$ for each $i$. In turn, we have $x - x'$ as a multiple of every $n_i$. With $n_i$ pairwise coprime, then $x - x'$ is a multiple of $N$. Simultaneously, recall that $x < N$, so $x - x' < N - x' < N$. It then follows that $0 \le x - x' < N$. However, $0$ is the only non-negative multiple of $N$ less than $N$, so $x - x' = 0$ and $x = x'$.

## A Custom Algorithm

Forturnately, the proof above asserts two notions. Specifically, existance shows we may find a number $x$ that satisfies a system of equations involving modular arthimetic, and uniqueness shows $x\ \mathrm{mod}\ N = t^{17}$, where $N$ is the product of moduli and $t$ is our plaintext in decimal form. With our proof, we may also construct an algorithm:

In [3]:
def crt(remainders, moduli):
    sum = 0
    N = 1
    
    for n_i in moduli:
        N *= n_i
    
    for r_i, n_i in zip(remainders, moduli):
        N_i = N // n_i
        M_i = pow(N_i, -1, n_i)
        sum += r_i * M_i * N_i
    
    return sum % N

Note that the built-in function $\texttt{pow}$ accepts three positional agruments: a base, a power, and a modulus. Here $\texttt{pow}$ uses the extended Euclidean algorithm to calculate $N_i^{-1}$ under modulus $n_i$, where $N_1^{-1}$ is an integer such that $N_i^{-1}N_i \equiv 1\ \ (\mathrm{mod}\ n_i)$. For example, $3^{-1}\equiv 2 \ \ (\mathrm{mod}\ 5)$ as $3\cdot2 \equiv 1\ \ (\mathrm{mod}\ 5)$. 

## Finding our Plaintext

Again, our algorithm will find an $x$ such that $x\ \mathrm{mod}\ N = t^{17}$, so we want to make $N$ large enough so that $0 \le x < N$, which will assert $x = t^{17}$ and make decryption significantly easier. In this may problem, we may find $t^{17}$ using as little as $5$ public keys. Although, it also takes very little time when we use all $100$:

In [4]:
t_17 = crt(ciphers, moduli)
print(t_17)

7939643493184983621421103302603742793403669458129357321975672938062573632922157007537684209333977705697211886123087737069529741187709448817983453703500422698664693979648504193680112404796864771063045963741689201305402926335227628552705763806005865506125363754693280193084421475666728649662346712485951778530052948475129127743238948425594395664886926457261217180308613991284272287255695131752811347546071413609516880389662807169606767863874914911519071091055542457223400834474139759869392416758263470869710548943704180610317315670615230410869672714587795591596166964100405617648384719998955088460547966589425103441685786763817125523481761032429563249949552892001967592786344055133909233136180677328499285594119059891012606340668312753921069101153735421011968596264757644875429163974990457519087593611587844445609799136673014092939496213027004391308215693574221023454718491643355286873273994165918809603294595411942006494837328034685498015392610213707700182567417374899401197191271434500282682333552069

Needless to say, $t^{17}$ is a very large number, so finding the $17^{\mathrm{th}}$ root would take quite some time if we tried a standard $\texttt{for}$ loop, iterating over the integers. Instead, we will use binary search to find our $t$:

In [5]:
def find_invpow(x,n):
    '''Uses binary search to find the integral n_th root of x'''
    high = 1
    while high ** n <= x:
        high *= 2
    low = high//2
    while low < high:
        mid = (low + high) // 2
        if low < mid and mid**n < x:
            low = mid
        elif high > mid and mid**n > x:
            high = mid
        else:
            return mid
    return mid + 1

t = find_invpow(t_17, 17)
print(t)

3338241147602646630929616677712790662799561818671257533591173576912964621664572526073572307837


Now, we just need to convert this number into readable text. To do this, we may use the $\texttt{long\_to\_bytes}$ method from $\texttt{Crypto.Util.number}$:

In [6]:
from Crypto.Util import number

number.long_to_bytes(t)

b'flag{M1gH7_W4N7_50M3_P4DD1nG_Nex7_t1m3}'

Alternatively, we may use the built-in $\texttt{to\_bytes}$ method:

In [7]:
from math import ceil, log

t.to_bytes(ceil(log(t, 2**8))) # or t.to_bytes((len(hex(t)) - 2)//2)

b'flag{M1gH7_W4N7_50M3_P4DD1nG_Nex7_t1m3}'